In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os

# Define paths for the processed datasets
processed_dir = r"C:\Users\ADMIN\Desktop\Projects\ETL-Data-Pipeline-for-Global-Superstore-Analytics\Dataset\processed_backup"
sales_data_path = os.path.join(processed_dir, "superstore_sales_backup.csv")
geolocation_data_path = os.path.join(processed_dir, "geolocation_backup.csv")

# Load the Superstore Sales and Geolocation datasets
print("Loading processed data...")
sales_data = pd.read_csv(sales_data_path)
geolocation_data = pd.read_csv(geolocation_data_path)
print("Data loaded successfully!")

Loading processed data...
Data loaded successfully!


In [3]:

# Display initial statistics and data preview
print("\nSales data statistics:")
print(sales_data.describe(include="all"))
print("\nGeolocation data statistics:")
print(geolocation_data.describe(include="all"))


Sales data statistics:
             Row ID        Order ID  Order Date   Ship Date       Ship Mode  \
count   9800.000000            9800        9800        9800            9800   
unique          NaN            4922        1230        1326               4   
top             NaN  CA-2018-100111  05/09/2017  26/09/2018  Standard Class   
freq            NaN              14          38          34            5859   
mean    4900.500000             NaN         NaN         NaN             NaN   
std     2829.160653             NaN         NaN         NaN             NaN   
min        1.000000             NaN         NaN         NaN             NaN   
25%     2450.750000             NaN         NaN         NaN             NaN   
50%     4900.500000             NaN         NaN         NaN             NaN   
75%     7350.250000             NaN         NaN         NaN             NaN   
max     9800.000000             NaN         NaN         NaN             NaN   

       Customer ID  Custome

In [13]:
# Data Cleaning: Handle missing values
print("\nHandling missing values...")

# Fill missing values in sales data
columns_to_fill = {"Profit": 0, "Discount": 0}  # Initialize columns to handle
if "Quantity" in sales_data.columns:  # Check if "Quantity" column exists
    columns_to_fill["Quantity"] = sales_data["Quantity"].mean()

sales_data.fillna(columns_to_fill, inplace=True)

# Drop rows with missing essential geolocation data
required_geolocation_columns = ["country", "city", "Region", "State"]
available_columns = [col for col in required_geolocation_columns if col in geolocation_data.columns]

if available_columns:  # Ensure columns are present before dropping
    geolocation_data.dropna(subset=available_columns, inplace=True)

# Validate missing values are resolved
print("\nMissing values after cleaning:")
print("Sales data missing values:")
print(sales_data.isnull().sum())
print("\nGeolocation data missing values:")
print(geolocation_data.isnull().sum())



Handling missing values...

Missing values after cleaning:
Sales data missing values:
Row ID            0
Order ID          0
Order Date        0
Ship Date         0
Ship Mode         0
Customer ID       0
Customer Name     0
Segment           0
Country           0
City              0
State             0
Postal Code      11
Region            0
Product ID        0
Category          0
Sub-Category      0
Product Name      0
Sales             0
dtype: int64

Geolocation data missing values:
iso-3166-1    0
country       0
city          0
dtype: int64


In [23]:
# Data Transformation: Create new calculated columns
print("\nAdding new calculated columns...")

# Check if the required columns exist in sales_data before creating calculated columns
if "Profit" in sales_data.columns and "Sales" in sales_data.columns:
    # Calculate profit margin percentage, capped between 0 and 100
    sales_data["Profit_Margin_Percent"] = (
        (sales_data["Profit"] / sales_data["Sales"] * 100)
        .replace([np.inf, -np.inf], 0)  # Handle divide-by-zero issues
        .clip(0, 100)  # Ensure values are within 0-100 range
    )
else:
    print("Warning: 'Profit' or 'Sales' column is missing. Cannot calculate Profit_Margin_Percent.")

# Sales category classification based on thresholds
if "Sales" in sales_data.columns:
    sales_data["Sales_Category"] = sales_data["Sales"].apply(
        lambda x: "High" if x > 1000 else ("Medium" if x > 500 else "Low")
    )
else:
    print("Warning: 'Sales' column is missing. Cannot calculate Sales_Category.")

# Display a preview of the transformed dataset
print("\nTransformed sales data preview:")
print(sales_data.head())



Adding new calculated columns...

Transformed sales data preview:
   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
1       2  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
2       3  CA-2017-138688  12/06/2017  16/06/2017    Second Class    DV-13045   
3       4  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   
4       5  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   

     Customer Name    Segment        Country             City       State  \
0      Claire Gute   Consumer  United States        Henderson    Kentucky   
1      Claire Gute   Consumer  United States        Henderson    Kentucky   
2  Darrin Van Huff  Corporate  United States      Los Angeles  California   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale

In [29]:
geolocation_data.columns

Index(['iso-3166-1', 'country', 'city'], dtype='object')

In [31]:
sales_data.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Sales_Category'],
      dtype='object')

In [41]:
import pandas as pd
import os

# Check column names to ensure they are as expected
print("Sales Data Columns:", sales_data.columns)
print("Geolocation Data Columns:", geolocation_data.columns)

# Strip any whitespace in column names to avoid issues
sales_data.columns = sales_data.columns.str.strip()
geolocation_data.columns = geolocation_data.columns.str.strip()

# Rename 'country' to 'Country' and 'city' to 'City' in geolocation_data
geolocation_data.rename(columns={'country': 'Country', 'city': 'City'}, inplace=True)

# Convert 'Country' and 'City' columns to string in both dataframes
sales_data['Country'] = sales_data['Country'].astype(str)
geolocation_data['Country'] = geolocation_data['Country'].astype(str)
sales_data['City'] = sales_data['City'].astype(str)
geolocation_data['City'] = geolocation_data['City'].astype(str)

# Perform the merge
merged_data = pd.merge(
    sales_data,
    geolocation_data,
    how="left",  # Left join to retain all sales data
    on=["Country", "City"]  # Now they should match
)

# Post-merge validation
print(f"\nMerged dataset contains {merged_data.shape[0]} rows and {merged_data.shape[1]} columns.")

# Check for missing values in the merged dataset
missing_after_merge = merged_data.isnull().sum()
print("\nMissing values in merged dataset (post-merge):")
print(missing_after_merge)


# Handle missing values post-merge
# Replace missing geolocation data with "Unknown" or other placeholders
columns_to_fill = {"Country": "Unknown", "City": "Unknown", "Latitude": 0.0, "Longitude": 0.0}
for column, fill_value in columns_to_fill.items():
    if column in merged_data.columns:
        merged_data[column] = merged_data[column].fillna(fill_value)  # Assign result back to column


# Validate missing values are resolved
print("\nMissing values after handling post-merge:")
print(merged_data.isnull().sum())

# Save the enriched data to the transformed folder
transformed_dir = r"C:\Users\ADMIN\Desktop\Projects\ETL-Data-Pipeline-for-Global-Superstore-Analytics\Dataset\transformed"
os.makedirs(transformed_dir, exist_ok=True)

transformed_data_path = os.path.join(transformed_dir, "transformed_data.csv")
merged_data.to_csv(transformed_data_path, index=False)

print("\nData transformation and enrichment completed. Transformed data saved successfully!")


Sales Data Columns: Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Sales_Category'],
      dtype='object')
Geolocation Data Columns: Index(['iso-3166-1', 'Country', 'City'], dtype='object')

Merged dataset contains 9800 rows and 20 columns.

Missing values in merged dataset (post-merge):
Row ID               0
Order ID             0
Order Date           0
Ship Date            0
Ship Mode            0
Customer ID          0
Customer Name        0
Segment              0
Country              0
City                 0
State                0
Postal Code         11
Region               0
Product ID           0
Category             0
Sub-Category         0
Product Name         0
Sales                0
Sales_Category       0
iso-3166-1        9800
dtype: int64

Missing values after handling 